Web -> Bedrock -> Translate -> Comprehend -> S3

1. Use `requests` to scrap web pages and extract the text using `BeautifulSoup`.

2. Send the text to Amazon Bedrock for cleaning/summarizing.

3. Translate to English using Amazon Translate.

4. Perform sentiment analysis using Amazon Comprehend.

5. Serialize the results and upload them to S3 as json files.

Before running, please run `pip install -r requirements.txt`.


In [91]:
import os
import json
import time
import hashlib
import logging
from typing import List, Tuple, Dict, Any

import requests
from bs4 import BeautifulSoup
import boto3
from botocore.exceptions import ClientError, BotoCoreError


In [92]:

# logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("web-pipeline")

# Environment variable configuration (can be done using AWS Configure or Environment Variables)
AWS_REGION = os.environ.get("AWS_REGION", "eu-west-1")
S3_BUCKET = os.environ.get("RESULT_S3_BUCKET", "ceu-jiaqi-2025")
BEDROCK_MODEL_ID = os.environ.get("BEDROCK_MODEL_ID", "mistral.mistral-7b-instruct-v0:2")

# boto3 clients
s3 = boto3.client("s3", region_name=AWS_REGION)
translate = boto3.client("translate", region_name=AWS_REGION)
comprehend = boto3.client("comprehend", region_name=AWS_REGION)
bedrock = boto3.client("bedrock-runtime", region_name=AWS_REGION)


In [93]:

# 1. Use `requests` to scrap web pages and extract the text using `BeautifulSoup`.


def fetch_page_text(url: str, timeout: int = 10) -> str:
    """
    Use requests + BeautifulSoup for simple text extraction.

    Strategy: First try the `<article>` tag; if none is found, merge all `<p>` paragraphs.

    Return to plain text (removing extra whitespace).
    """
    headers = {"User-Agent": "Mozilla/5.0 (compatible; web-pipeline/1.0)"}
    resp = requests.get(url, timeout=timeout, headers=headers)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    # Try common text containers.
    candidates = []
    article = soup.find("article")
    if article:
        candidates.append(article.get_text(separator="\n", strip=True))

    # Common class names (simplified)
    for cls in ("main", "content", "post", "article-body", "entry-content"):
        el = soup.find(class_=cls)
        if el:
            candidates.append(el.get_text(separator="\n", strip=True))

    # fallback: merge all `<p>` paragraphs
    if not candidates:
        paragraphs = soup.find_all("p")
        text = "\n\n".join(p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True))
    else:
        # Select the longest candidate.
        text = max(candidates, key=len)

    # Clean up extra whitespace
    text = "\n".join(line.strip() for line in text.splitlines() if line.strip())
    return text


In [94]:

# 2. Send the text to Amazon Bedrock for cleaning/summarizing.

def bedrock_clean_text(text: str, model_id: str = BEDROCK_MODEL_ID, timeout_seconds: int = 60) -> str:
    """
    If `BEDROCK_MODEL_ID` is configured, the `text` is sent to the Bedrock model, which then returns the cleaned text.
    If `model_id` is not configured, the original text is returned directly.
    """
    if not model_id:
        logger.info("No Bedrock model configured, skipping Bedrock step.")
        return text

    prompt = (
        "[INST]You are a text-cleaning assistant. "
        "Given a noisy HTML-extracted article text, return a cleaned, readable plaintext article. "
        "Do not include any introductory phrases, just return the cleaned text."
        "\n\nARTICLE:\n" + text + "[/INST]"
    )

    body = json.dumps({
        "prompt": prompt,
        "max_tokens": 4096, 
        "temperature": 0.1,
        })

    
    try:
        response = bedrock.invoke_model(
            modelId=model_id,
            contentType="application/json",
            accept="application/json",
            body=body.encode("utf-8")
        )
        body_bytes = response["body"].read()
        response_json = json.loads(body_bytes.decode("utf-8"))
        cleaned = response_json['outputs'][0]['text']
        return cleaned.strip()
    except (ClientError, BotoCoreError) as e:
        logger.exception("Bedrock invocation failed, returning original text: %s", e)
        return text


In [95]:
# 3. Segment long texts (because Translate/Comprehend has length limitations).

def split_text_chunks(text: str, max_chars: int = 4500) -> List[str]:
    """
    Segment by paragraph and combine into blocks not exceeding max_chars=4500.
    """
    paragraphs = [p.strip() for p in text.split('\n\n') if p.strip()]
    chunks = []
    cur = []
    cur_len = 0
    for p in paragraphs:
        if len(p) > max_chars:
            # If a single paragraph is too long, then it can be split into sentences or fixed lengths.
            for i in range(0, len(p), max_chars):
                piece = p[i:i+max_chars]
                if cur:
                    chunks.append('\n\n'.join(cur))
                    cur = []
                    cur_len = 0
                chunks.append(piece)
        else:
            if cur_len + len(p) + 2 > max_chars:
                chunks.append('\n\n'.join(cur))
                cur = [p]
                cur_len = len(p)
            else:
                cur.append(p)
                cur_len += len(p) + 2
    if cur:
        chunks.append('\n\n'.join(cur))
    return chunks


In [96]:
# 4. Translation (send each chunk to Translate, then reassemble them)

def translate_chunks_to_english(chunks: List[str]) -> Tuple[str, str]:
    """Translate each text block into English and merge them, returning (full_translated_text, detected_source_language). 
    SourceLanguageCode is adjusted manually.
    """
    translated_parts = []
    detected_lang = None
    for chunk in chunks:
        try:
            resp = translate.translate_text(Text=chunk, SourceLanguageCode="it",TargetLanguageCode="en")
            translated_parts.append(resp.get("TranslatedText", ""))
            if detected_lang is None:
                detected_lang = resp.get("SourceLanguageCode")
        except (ClientError, BotoCoreError) as e:
            logger.exception("Translate failed for a chunk, adding original chunk instead: %s", e)
            translated_parts.append(chunk)
    return "\n\n".join(translated_parts), (detected_lang or "unknown")


In [97]:

# 5. Sentiment analysis (using Comprehend on translated English text)

def analyze_sentiment_for_text(text: str, language_code: str = "en") -> Dict[str, Any]:
    """If the text is too long, call `detect_sentiment` block by block, then merge the scores.
    Returns the overall sentiment label (majority or score-based synthesis) and details for each block.
    """
    chunks = split_text_chunks(text, max_chars=4500)
    results = []
    # Comprehend detect_sentiment processes a piece of text each time.
    for chunk in chunks:
        try:
            resp = comprehend.detect_sentiment(Text=chunk, LanguageCode=language_code)
            results.append(resp)
        except (ClientError, BotoCoreError) as e:
            logger.exception("Comprehend detect_sentiment failed for a chunk: %s", e)
            results.append({"Sentiment": "ERROR", "SentimentScore": {}})

    # Merging logic: Averaging of all sentiment scores
    score_sum = {"Positive": 0.0, "Negative": 0.0, "Neutral": 0.0, "Mixed": 0.0}
    valid = 0
    for r in results:
        sc = r.get("SentimentScore") or {}
        if sc:
            valid += 1
            for k in score_sum.keys():
                score_sum[k] += float(sc.get(k, 0.0))
    if valid > 0:
        avg_score = {k: v / valid for k, v in score_sum.items()}
        # The sentiment with the highest average score was selected as the overall sentiment.
        overall_sentiment = max(avg_score.items(), key=lambda x: x[1])[0]
    else:
        avg_score = {}
        overall_sentiment = "UNKNOWN"

    return {
        "overall_sentiment": overall_sentiment,
        "average_scores": avg_score,
        "per_chunk": results,
    }


In [98]:
# 6. Upload JSON to S3

def upload_json_to_s3(obj: Dict[str, Any], key: str) -> str:
    body = json.dumps(obj, ensure_ascii=False, indent=None).encode("utf-8")
    try:
        s3.put_object(Bucket=S3_BUCKET, Key=key, Body=body, ContentType="application/json; charset=utf-8")
    except ClientError:
        logger.exception("Failed to upload result to S3")
        raise
    return f"s3://{S3_BUCKET}/{key}"


In [99]:
# 7. String all the steps together into a single `process_url` function.

def process_url(url: str, do_bedrock_clean: bool = True) -> Dict[str, Any]:
    start = time.time()
    logger.info("Processing URL: %s", url)
    raw_text = fetch_page_text(url)
    if not raw_text or len(raw_text.strip()) == 0:
        logger.warning("No textual content extracted from %s", url)
        return {"url": url, "status": "no_content"}

    cleaned_text = raw_text
    if do_bedrock_clean and BEDROCK_MODEL_ID:
        cleaned_text = bedrock_clean_text(raw_text, model_id=BEDROCK_MODEL_ID)

    chunks = split_text_chunks(cleaned_text)
    translated_text, src_lang = translate_chunks_to_english(chunks)


    sentiment = analyze_sentiment_for_text(translated_text, language_code="en")

    result = {
        "source_url": url,
        "detected_source_language": src_lang,
        "raw_text_excerpt": raw_text[:2000],
        "cleaned_text_excerpt": cleaned_text[:2000],
        "translated_text_excerpt": translated_text[:2000],
        "sentiment": sentiment,
        "meta": {
            "char_counts": {
                "raw": len(raw_text),
                "cleaned": len(cleaned_text),
                "translated": len(translated_text),
            },
            "processing_time_sec": time.time() - start,
        }
    }

    # Use the URL's SHA1 hash + timestamp as the object key.
    key = f"results/{hashlib.sha1(url.encode()).hexdigest()}_{int(time.time())}.json"
    s3_uri = upload_json_to_s3(result, key)
    result["s3_uri"] = s3_uri
    logger.info("Finished %s -> %s", url, s3_uri)
    return result


In [ ]:
# 8. Demo: Handling single or multiple URLs

if __name__ == "__main__":
    # Put a list of URLs to be processed into urls.
    urls = [
        "https://www.eurosport.it/tennis/caso-doping-sinner-la-sentenza-spiegata-bene-le-date-i-dettagli-e-come-e-avvenuta-lassunzione-del-clostebol_sto20030540/story.shtml",
        ]
    for u in urls:
        try:
            out = process_url(u, do_bedrock_clean=True) 
            print(json.dumps(out, ensure_ascii=False, indent=2))
        except Exception as e:
            logger.exception("Error processing %s: %s", u, e)


INFO:web-pipeline:Processing URL: https://www.eurosport.it/tennis/caso-doping-sinner-la-sentenza-spiegata-bene-le-date-i-dettagli-e-come-e-avvenuta-lassunzione-del-clostebol_sto20030540/story.shtml
INFO:web-pipeline:Finished https://www.eurosport.it/tennis/caso-doping-sinner-la-sentenza-spiegata-bene-le-date-i-dettagli-e-come-e-avvenuta-lassunzione-del-clostebol_sto20030540/story.shtml -> s3://ceu-jiaqi-2025/results/92da28aa0fbdd2174f1113e1c2cbc72278334d59_1765468979.json
INFO:web-pipeline:Processing URL: https://www.rainews.it/articoli/2024/08/sport-tennis-atp-jannik-sinner-positivo-doping-in-aprile-assolto-per-assunzione-inconsapevole-8024a639-1822-4621-87a9-99381b443660.html


{
  "source_url": "https://www.eurosport.it/tennis/caso-doping-sinner-la-sentenza-spiegata-bene-le-date-i-dettagli-e-come-e-avvenuta-lassunzione-del-clostebol_sto20030540/story.shtml",
  "detected_source_language": "it",
  "raw_text_excerpt": "Jannik Sinner en Indian Wells\nCredit Foto Getty Images\nJannik Sinner en Indian Wells\nCredit Foto Getty Images\nCom'è stata la tua esperienza oggi?\nCambia versione:",
  "cleaned_text_excerpt": "Jannik Sinner at Indian Wells\nCredit: Getty Images\n\nExperience today? (Change version:)",
  "translated_text_excerpt": "Jannik Sinner at Indian Wells\nCredit: Getty Images\n\nExperience today? (Change version:)",
  "sentiment": {
    "overall_sentiment": "Neutral",
    "average_scores": {
      "Positive": 0.01025056466460228,
      "Negative": 0.00014862230455037206,
      "Neutral": 0.9895944595336914,
      "Mixed": 6.414672043320024e-06
    },
    "per_chunk": [
      {
        "Sentiment": "NEUTRAL",
        "SentimentScore": {
          "Positi

INFO:web-pipeline:Finished https://www.rainews.it/articoli/2024/08/sport-tennis-atp-jannik-sinner-positivo-doping-in-aprile-assolto-per-assunzione-inconsapevole-8024a639-1822-4621-87a9-99381b443660.html -> s3://ceu-jiaqi-2025/results/c26e5d1c06210d9256db06fd8a1183041b347a13_1765468988.json
INFO:web-pipeline:Processing URL: https://www.avvenire.it/agora/sport/positivo-al-doping-ma-innocente-lultima-disavventura-di-sinner_79977


{
  "source_url": "https://www.rainews.it/articoli/2024/08/sport-tennis-atp-jannik-sinner-positivo-doping-in-aprile-assolto-per-assunzione-inconsapevole-8024a639-1822-4621-87a9-99381b443660.html",
  "detected_source_language": "it",
  "raw_text_excerpt": "Sport\nTennis\nATP\nLo rende noto il suo ufficio stampa\nSinner positivo al doping in aprile, ma assolto per \"assunzione inconsapevole”\nDurante gli Indian Wells il suo fisioterapista aveva usato uno spray contenente clostebol per medicare una piccola ferita. Al tennista sono stati tolti 400 punti, ma resta il numero 1 del mondo e può continuare a giocare\n21/08/2024\nJannik Sinner è risultato positivo al test antidoping quattro mesi fa, in aprile, ma un’indagine indipendente della Tennis Integrity Agency (Itia), l'agenzia mondiale sul doping, ha fatto cadere le accuse contro di lui per un’”assunzione inconsapevole”. Il problema sarebbe stato causato da uno spray utilizzato dal fisioterapista. Lo fa sapere l'ufficio stampa del campio

INFO:web-pipeline:Finished https://www.avvenire.it/agora/sport/positivo-al-doping-ma-innocente-lultima-disavventura-di-sinner_79977 -> s3://ceu-jiaqi-2025/results/fe0b692fa3c9f1b1b1bb15391c1cd035bfa64127_1765468993.json
INFO:web-pipeline:Processing URL: https://www.corrieredellosport.it/news/tennis/sinner/2025/02/15-138430669/sinner_e_la_vicenda_doping_tutte_le_tappe_di_una_vicenda_infinita


...

{
  "source_url": "https://www.today.it/sport/sinner-positivo-doping-allenatore-spray-fisioterapista-giacomo-naldi.html",
  "detected_source_language": "it",
  "raw_text_excerpt": "VIDEO DEL GIORNO\nMasterChef, l’intervista ai giudici: “È l’unico programma vero della Tv. Ecco l’errore imperdonabile nei piatti”\ncaso doping\nSinner positivo al test antidoping, parla l'allenatore: \"Non avevo mai visto quello spray\"\nIl 23enne numero uno al mondo è stato trovato positivo  al Clostebol, uno steroide. La sostanza è contenuta in uno spray che il suo fisioterapista avrebbe usato per medicargli una ferita\nRedazione\n21 agosto 2024 13:27\nFacebook\nWhatsApp\nCondividi\nAd alcune ore dallo scoppio del\ncaso doping\nsu Jannik\nSinner\n, risultato positivo ad alcuni test effettuati ad aprile, ma ritenuto innocente dall'Agenzia internazionale per l'integrità del tennis, è l'allenatore dell'atleta a parlare: \"La verità è venuta a galla, Jannik non ha avuto colpe o negligenze, e spero che possa s

INFO:web-pipeline:Finished https://www.startmag.it/sanita/cose-la-sostanza-dopante-alla-quale-e-risultato-positivo-sinner/ -> s3://ceu-jiaqi-2025/results/6ce34b2a06eb08f1f995aaa1015afba378cf3290_1765469016.json
INFO:web-pipeline:Processing URL: https://www.tuttosport.com/news/tennis/2024/08/20-131691747/sinner_positivo_all_antidoping_ma_innocente_tutta_la_verita_sul_clostebol


{
  "source_url": "https://www.startmag.it/sanita/cose-la-sostanza-dopante-alla-quale-e-risultato-positivo-sinner/",
  "detected_source_language": "it",
  "raw_text_excerpt": "DI\nMarco Dell'Aguzzo\nIn cosa consiste l’offerta di Flacks per salvare l’ex Ilva (senza licenziamenti)\nIn gara per Acciaierie d’Italia sono rimasti solo due fondi americani: uno è Flacks Group …",
  "cleaned_text_excerpt": "Marco Dell'Aguzzo: What Flacks Offers to Save Ilva (Without Layoffs)\nTwo American funds remain in the race for Acciaierie d'Italia: one is Flacks Group.",
  "translated_text_excerpt": "Marco Dell'Aguzzo: What Flacks Offers to Save Ilva (Without Layoffs)\nTwo American funds remain in the race for Acciaierie d'Italia: one is Flacks Group.",
  "sentiment": {
    "overall_sentiment": "Neutral",
    "average_scores": {
      "Positive": 0.001327738631516695,
      "Negative": 0.012050510384142399,
      "Neutral": 0.9866158962249756,
      "Mixed": 5.87017711950466e-06
    },
    "per_chunk": [


INFO:web-pipeline:Finished https://www.tuttosport.com/news/tennis/2024/08/20-131691747/sinner_positivo_all_antidoping_ma_innocente_tutta_la_verita_sul_clostebol -> s3://ceu-jiaqi-2025/results/3ad2f56b6056516b2991062a6673f9a92c90e191_1765469028.json


{
  "source_url": "https://www.tuttosport.com/news/tennis/2024/08/20-131691747/sinner_positivo_all_antidoping_ma_innocente_tutta_la_verita_sul_clostebol",
  "detected_source_language": "it",
  "raw_text_excerpt": "Jannik Sinner è risultato positivoa tracce di un metabolita della sostanza Clostebol (meno di un miliardesimo di grammo). Lo rende noto l'International Tennis Integrity Agency (Itia) in un comunicato. \"Dopo un'indagine approfondita, l'Itia e Jannik hanno scoperto che lacontaminazione involontaria di Clostebol è avvenuta tramite il trattamento ricevuto dal suo fisioterapista. Il suo personal trainer ha acquistato un prodotto, facilmente reperibile senza ricetta in qualsiasi farmacia italiana, che ha dato al fisioterapista di Jannik per curare un taglio sul dito del fisioterapista. Jannik non ne sapeva nulla e il suo fisioterapista non sapeva che stava usando un prodotto contenente Clostebol.Il fisioterapista ha curato Jannik senza guantie, insieme a varie lesioni cutanee sul 